In [1]:
import xmltodict
import json
import pandas as pd

# with open('consol.xml', 'r') as f:
with open('consolidated.xml', 'r') as f:
    data = f.read()

consol_dict = xmltodict.parse(data)

consol_json = json.dumps(consol_dict, indent=2)

print(consol_json[:8000])

{
  "CONSOLIDATED_LIST": {
    "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "@xsi:noNamespaceSchemaLocation": "https://www.un.org/sc/resources/sc-sanctions.xsd",
    "@dateGenerated": "2022-10-21T20:40:12.605-04:00",
    "INDIVIDUALS": {
      "INDIVIDUAL": [
        {
          "DATAID": "6908555",
          "VERSIONNUM": "1",
          "FIRST_NAME": "RI",
          "SECOND_NAME": "WON HO",
          "THIRD_NAME": null,
          "UN_LIST_TYPE": "DPRK",
          "REFERENCE_NUMBER": "KPi.033",
          "LISTED_ON": "2016-11-30",
          "COMMENTS1": "Ri Won Ho is a DPRK Ministry of State Security Official stationed in Syria supporting KOMID.",
          "DESIGNATION": {
            "VALUE": "DPRK Ministry of State Security Official"
          },
          "NATIONALITY": {
            "VALUE": "Democratic People's Republic of Korea"
          },
          "LIST_TYPE": {
            "VALUE": "UN List"
          },
          "LAST_DAY_UPDATED": {
            "VALUE":

In [2]:
indivs = consol_dict['CONSOLIDATED_LIST']['INDIVIDUALS']['INDIVIDUAL']
print(type(indivs))

df_norm = pd.json_normalize(indivs,
                                    meta=[
                                          ['INDIVIDUAL_ADDRESS', 'COUNTRY', 'STATE_PROVINCE','ZIP_CODE', 'NOTE'],
                                          ['INDIVIDUAL_DOCUMENT', 'TYPE_OF_DOCUMENT', 'NUMBER', 'ISSUING_COUNTRY', 'NOTE'],
                                    ['INDIVIDUAL_DATE_OF_BIRTH', 'TYPE_OF_DATE', 'DATE'],
                                    ['INDIVIDUAL_ALIAS', 'QUALITY', 'ALIAS_NAME'],
                                     ['INDIVIDUAL_PLACE_OF_BIRTH','CITY', 'STATE_PROVINCE', 'COUNTRY']
                                          ]                                   
                                   )  #, record_path =['INDIVIDUAL'])
# df_norm


<class 'list'>


In [3]:
# pd.set_option('display.expand_frame_repr', True)
# pd.set_option('max_colwidth', -1)
import re
rstring = "(\([a-z]{3,4}.\d{3,4}\))"
df_norm['TARGETS'] = df_norm.loc[:,'COMMENTS1'].str.findall(r'\([a-z]{3,4}.\d{3,4}\)', flags=re.IGNORECASE)
df_norm['CONN_COUNT'] = df_norm['TARGETS'].str.len()
# print(df_norm.loc[(df_norm['TARGETS'].notnull()) & (df_norm['CONN_COUNT']>1)],['TARGETS','CONN_COUNT'])



In [4]:

# df_norm.rename(str.replace('INDIVIDUAL_',''), axis='columns')

df_norm.columns = [c.replace("INDIVIDUAL_", "") for c in list(df_norm.columns)]
# print(df_norm.columns)
# df_norm.info()

# display(dataFrame.loc[(dataFrame['Salary']>=100000) & (dataFrame['Age']< 40) & (dataFrame['JOB'].str.startswith('D')),
# print(df_norm[[(df_norm.CONNECTIONS.notnull()) and ('CONN_COUNT'>1)]][['CONNECTIONS', 'COMMENTS1']])  # [['CONNECTIONS' == None]]
# https://regex101.com/r/ejCzvg/1
# [itm for itm in S.str.extractall('(.*)') if (itm is not None and len(str(itm))>0)]
# [itm[0] for itm in S.str.findall('(.*)') if (itm is not None and len(itm)>0)]
# [itm[0] for itm in S.str.findall('^[Ff].*') if (itm is not None and len(itm)>0)]

In [5]:
col_list = list(df_norm.columns)
col_list.remove('TARGETS')
# for col in col_list:
#     print(df_norm[col].value_counts())


In [23]:
# graph[df_norm['REFERENCE_NUMBER']] = df_norm['TARGETS']
target_list = df_norm['TARGETS']
ref_no = df_norm['REFERENCE_NUMBER']
ref_no.shape
link_list = []

for i in range(ref_no.shape[0]):
    source = ref_no.loc[i]
    if (target_list.loc[i] is not None) and (len(target_list.loc[i]) > 0):        
        # print(i, target_list.loc[i])
        targets2 = []
        for target in target_list.loc[i]:
            target = target.replace('(','').replace(')','')
            if source == target:
                print('WARNING', source, ' equals ', target)
            targets2.append(target)
            link2 = [ref_no.loc[i], target]
            link_list.append(link2)
            target_list.loc[i] = targets2            

print(len(link_list))

df_edges = pd.DataFrame(link_list, columns=['Source', 'Target'] )
df_edges.to_csv('un_indiv_edges.csv', index=False)
graph = dict(zip(df_norm.REFERENCE_NUMBER, df_norm.TARGETS))
print(len(graph.keys()))
print(graph)



340
703
{'KPi.033': [], 'KPi.037': None, 'KPi.038': None, 'QDi.430': ['QDe.115'], 'KPi.034': [], 'KPi.035': [], 'KPi.036': [], 'KPi.030': [], 'KPi.032': [], 'KPi.031': [], 'KPi.039': None, 'YEi.001': [], 'SOi.020': [], 'QDi.400': ['QDe.137'], 'QDi.343': ['QDe.004', 'QDe.137', 'QDe.115'], 'IRi.001': [], 'TAi.155': ['TAe.012', 'TAi.144'], 'QDi.012': ['QDe.098'], 'IQi.043': None, 'QDi.192': ['QDe.003'], 'TAi.121': [], 'TAi.024': [], 'TAi.128': [], 'TAi.100': [], 'QDi.054': [], 'TAi.130': [], 'TAi.162': ['TAe.010', 'TAi.163'], 'QDi.018': [], 'TAi.142': [], 'TAi.105': ['TAi.144'], 'TAi.080': [], 'QDi.295': ['QDe.092', 'QDi.086'], 'QDi.229': ['QDe.011'], 'TAi.008': [], 'TAi.145': ['TAi.024'], 'QDi.309': ['QDe.004', 'QDe.130', 'QDe.019', 'QDe.121'], 'QDi.363': ['QDe.115'], 'QDi.086': [], 'QDi.414': ['QDe.115'], 'TAi.160': [], 'TAi.167': ['TAi.160'], 'QDi.304': ['QDe.133', 'QDi.217', 'QDi.293', 'QDe.092'], 'CFi.002': [], 'SOi.018': [], 'MLi.004': ['QDe.135'], 'MLi.005': ['QDi.316'], 'QDi.316':

/var/folders/z7/t8qyfffs7h3dqbzjkmmhz03r0000gp/T/ipykernel_80387/2981508184.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_list.loc[i] = targets2


In [7]:
# S=pd.Series(['Finland','Colombia','Florida','Japan','Puerto Rico','Russia','france'])
# [itm[0] for itm in S.str.findall('^[Ff].*') if len(itm)>0]


In [8]:
# Add Preprocessors
# feature_eng_ip_pair_cnt = GroupByCountPreprocessor(func_kwargs={"resulting_column_name":"ip_pair_count",
#         "groupby_list":["id_orig_h","id_resp_h"],
#         "count_key":"uid"})
# self.output_obj[kwargs["resulting_column_name"]] = \
#     input_obj.groupby(
#         kwargs["groupby_list"])[kwargs["count_key"]].transform('count')